# Visualising the data

In this notebook, we will guide you through visualising the data you downloaded through the API in the form of graphs. In the process we introduce ```pandas```, a very popular data processing library. Entire books have been written on the  ```pandas``` library, and there is an extensive online [documentation](https://pandas.pydata.org/docs/); in this notebook, we will restrict ourselves to the minimum required to use the library as a wrapper for ```matplotlib```, a lower-level data visualisation library.

Both ```pandas``` and ```matplotlib``` are installed on the QMUL JupyterHub server, and they are likely to be part of any major Python distribution, so the following ```import``` statements should just work. Installation instructions for [pandas](https://pandas.pydata.org/docs/getting_started/install.html) and [matplotlib](https://matplotlib.org/users/installing.html) are available online.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import json

In [ ]:
# an iPython  "magic" that enables the embedding of matplotlib output
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Time series data

We start by loading the data from the JSON file in which we saved it. Reading a JSON file in Python is not any more difficult than writing it:

In [ ]:
with open("admissions.json", "rt") as INFILE:
    admissions=json.load(INFILE)
with open("cases.json", "rt") as INFILE:
    cases=json.load(INFILE)
with open("deaths.json", "rt") as INFILE:
    deaths=json.load(INFILE)

### Wrangling the data

Whatever your query, the data is unlikely to come in a form that's amenable to be plotted directly; some wrangling, munging, or other bit of cool data science jargon is required. Our aim is to extract values for the *x* axis of our plot (in this case the dates). These will become the *index* of a ```DataFrame```, of which the *columns* will contain the data proper. You can, for the moment, think of a ```DataFrame``` as the Python equivalent of an Excel spreadsheet.

In [ ]:
cases[:5]

As we can see, the data consists of a list of dictionaries; actual data is stored in the inner dictionaries under the ```date``` and ```metric_value``` keys. Let us start by retrieving those values and bringing everything together into a dictionary with the dates as keys, and inner dictionaries with the three metrics as values:

In [ ]:
data={}
for dataset in [admissions, cases, deaths]:
    for entry in dataset:
        date=entry['date']
        metric=entry['metric']
        value=entry['metric_value']
        if date not in data:
            data[date]={}
        data[date][metric]=value

In [ ]:
data

Note that some of the metrics are missing for some dates, and the keys ate not sorted by date, but by insertion:

In [ ]:
data['2020-01-30']

We then want to extract all the dates and sort them. Luckily, with dates written year-first, alphabetical ordering does the trick.

In [ ]:
dates=list(data.keys())
dates.sort()
dates

Do not assume this is a proper time series: some dates might be missing. At any rate, we find the earliest and latest date and convert them to the ```pandas``` type for representing dates.

In [ ]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [ ]:
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])
print (startdate, ' to ', enddate)

We are now ready to define our ```DataFrame```. To start with, we create an index as a ```date_range```: this is the date analog of a ```range``` for integers, and it will include any dates that may be missing from our list.
We then proceed to define the ```DateFrame``` by specifying its index and the title of its columns.

In [ ]:
index=pd.date_range(startdate, enddate, freq='D')
timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'admissions', 'deaths'])
timeseriesdf

As you can see, the ```DataFrame``` resembles a spreadsheet, or if you want a matrix with labels. We now proceed to fill it in with values from our three statistics. Note that there are many ways of filling in a ```DataFrame```: this is probably not the most efficient, but it is in my view the easiest to understand and one of those that give you more control over the values you enter.

In [ ]:
# translate the columns to our metrics
metrics ={'cases': 'COVID-19_cases_casesByDay',
          'admissions': 'COVID-19_healthcare_admissionByDay',
          'deaths': 'COVID-19_deaths_ONSByDay'}

for date, entry in data.items(): # each entry is a dictionary with cases, admissions and deaths
    pd_date=parse_date(date) # convert to Pandas format
    for column in ['cases', 'admissions', 'deaths']: 
        metric_name=metrics[column]
        # do not assume all values are there for every date - if a value is not available, insert a 0.0
        value= entry.get(metric_name, 0.0)
        # this is the way you access a specific location in the dataframe - use .loc
        # and put index,column in a single set of [ ]
        timeseriesdf.loc[date, column]=value
            
# fill in any remaining "holes" due to missing dates
timeseriesdf.fillna(0.0, inplace=True)
            
timeseriesdf

### Plotting

Creating a basic plot of our data is now dead easy. We just need to call the ```plot()``` method on our ```DataFrame```; this will call the underlying ```matplot.pyplot``` functions to create a plot of the data. For more options and fancier graphs, see the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html).

In [ ]:
ax=timeseriesdf.plot() # easy peasy...
ax.set_title('Daily cases, hospital admissions and deaths');

After lording it for centuries over engineering calculations up to the advent of computers, logarithms had never been so popular as they have become since the rise of Covid dashboards - now everybody and their uncle knows what a log scale is. There is a reason for that: exponential phenomena are well suited for plotting on log scales, where an exponential propagation becomes a straight line. You wouldn't want to miss out on this centuries-old fad:

In [ ]:
ax=timeseriesdf.plot(logy=True) # ...lemon squeezy
ax.set_title('Daily cases, hospital admissions and deaths (log scale)');

## Lineage data

As a second example, let's lineage data. The data scructure, as you can see, is a bit different, so this will be another good exercise in data wrangling.

In [ ]:
with open("lineage.json", "rt") as INFILE:
    lineage=json.load(INFILE)

In [ ]:
lineage[:5]

### Data munging



Once again, there are no fixed rules for this - just see in what format your data comes, have clear in your mind what you are aiming for, and wrangle your data accordingly. In this case, we again want to get to a ```DataFrame``` with the dates as the *index*, and the prevalence of the different variants as its *columns*.

In this case, the fields of interest out of this series are ```date```, ```stratum``` and ```metric_value```. Notice also that for each ```date``` there are several entries, one for each variant represented in that period. Let us again try to organise the data into a dictionary of dictionaries, with the dates as keys:

In [ ]:
data={}
for entry in lineage:
    date=entry['date']
    variant=entry['stratum']
    value=entry['metric_value']
    if date not in data:
        data[date]={}
    data[date][variant]=value

In [ ]:
data 

Extracting the dates works as above. Notice that the dates are at least one week apart, as this is a weekly metric; in fact, they are all Mondays.

In [ ]:
dates=list(data.keys())
dates.sort()
dates

In [ ]:
startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])
print (startdate, ' to ', enddate)

Finding the columns is a bit more complicated, as we do not know their names and they do not all appear at the same time:

In [ ]:
variants=[]
for entry in data.values():
    for var in entry.keys():
        if var not in variants:
            variants.append(var)
variants.sort()
print(variants)

We are finally ready to define our ```DataFrame```. We use a date range as an *index*, but this time we have to create with a weekly spacing. The sorted variants will be the columns.

In [ ]:
# W-MON is "weekly on Mondays"
index=pd.date_range(startdate, enddate, freq='W-MON')
lineagedf=pd.DataFrame(index=index, columns=variants)
lineagedf

Filling in ```lineagedf``` is only a matter of looping over the dates and filling in the columns with the corresponding values.

In [ ]:
for date, entry in data.items(): # each entry is a dic with some variants and values
    pd_date=parse_date(date) # convert to Pandas format
    for column in entry.keys(): # the variants
        # this is the way you access a specific location in the dataframe - use .loc
        # and put index,column in a single set of [ ]
        lineagedf.loc[date, column]=entry[column]
            
# fill in any remaining "holes" due to missing dates and variants
lineagedf.fillna(0.0, inplace=True)
            
lineagedf

### Plotting

We plot the data as a multiple stacked bar plot, aggregated by quarter. Each bar has the same meaning as a pie chart, representing proportions, and we have a series them arranged by date. For this and fancier bar plots (and other plots), see the excellent [book](https://link.springer.com/book/10.1007/978-3-031-57051-3) on data visualisation by F. Bianconi. This [tutorial](https://www.shanelynn.ie/bar-plots-in-python-using-pandas-dataframes/) is also useful.

To begin with, we group the dataset by quarters by averaging, in order to reduce the amount of data and at the same time deal with missing data:

In [ ]:
# 1Q means "by quarter"; compute the average row
quarterly= lineagedf.groupby(pd.Grouper(freq='1Q')).mean() 
# since some data are missing, the sum across an average row may be less than one
totals=quarterly.sum(axis=1) # sum over the rows
# make sure it's all normalised to 100
quarterly=quarterly.div(totals, axis=0)*100 # divide the columns by the totals
# reverse the rows for the graph, so older dates will be on top
quarterly = quarterly[::-1]

Plotting is now relatively straightforward:

In [ ]:
# we need a colormap with more than 11 colours 
ax=quarterly.plot(kind='barh', stacked=True,cmap='tab20')
ax.legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
# format the dates nicely
ax.set_yticklabels(quarterly.index.strftime('%Y-%m-%d'))
ax.set_title('Lineage prevalence by quarter ending');

## How to get ~~yourself~~ your data into a pickle (file)

In the actual dashboard, at this point, there is no need to save the dataframes - you can just add interactive controls to the graphs as you plot them. While developing, however, we want to save the wrangled data in order to be able to experiment with the controls. Pandas luckily makes it easy by providing a method that saves a ```DataFrame``` to a "pickle" file. This is based on the ```pickle``` [library](https://docs.python.org/3/library/pickle.html), that however is imported by ```pandas```, so you do not need to import it yourself. While JSON is a standard format that is language-independent, a *pickle* file is Python-specific. This format is thus more flexible, allowing the [serialisation](https://en.wikipedia.org/wiki/Serialization) of a broader range of Python objects; however, it is less portable.

In [ ]:
# pandas makes saving to a pickle file dead easy:
timeseriesdf.to_pickle("timeseriesdf.pkl")
lineagedf.to_pickle("lineagedf.pkl")

## Your turn

Load the data you saved in the JSON files, munge it, turn it into a ```DataFrame``` and display it; then save your ```DataFrame``` to a *pickle* file. In the next notebook, when we will show you how to add interactive controls to your graphs. 

**(C) 2020,2024 Fabrizio Smeraldi** ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).